### Tensors in PyTorch

Tensors are specialized data structures used in PyTorch to represent model inputs, outputs, and parameters. While they are conceptually similar to arrays and matrices, they offer additional features such as support for hardware accelerators like GPUs and automatic differentiation.

In [1]:
import os
# The jupyter notebook is launched from your $HOME directory.
# Change the working directory to the workshop directory
# which was created in your username directory under /scratch/vp91
os.chdir(os.path.expandvars("/scratch/vp91/$USER/"))

### Creating a Tensor

In [2]:
import torch
import numpy as np

##### 1. Directly from data

In [26]:
data = [[1, 2],[3, 4]]
x_tensor= torch.tensor(data)

##### 2. From NumPy

In [27]:
x_np = np.array(data)
x_tensor = torch.from_numpy(x_np)

##### 3. From another Tensor

**torch.rand_like()** returns a tensor with the same size as input that but filled with random numbers from the interval [0,1).

In [11]:
x_tensor = torch.ones_like(x_tensor)
y_tensor = torch.rand_like(x_tensor, dtype=torch.float) 
print(x_tensor)

tensor([[1, 1],
        [1, 1]])


### Operations on Tensors

#### 1. indexing and slicing

In [13]:
x_tensor = torch.ones(4, 4)
print(f"First row: {x_tensor[0]}")
print(f"First column: {x_tensor[:, 0]}")
print(f"Last column: {x_tensor[..., -1]}")
x_tensor[:,1] = 0
print(x_tensor)

First row: tensor([1., 1., 1., 1.])
First column: tensor([1., 1., 1., 1.])
Last column: tensor([1., 1., 1., 1.])
tensor([[1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.]])


#### 2. Concatenate multiple tensors

In [15]:
y_tensor = torch.cat([x_tensor, x_tensor, x_tensor], dim=1)
print(y_tensor)

tensor([[1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1.],
        [1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1.],
        [1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1.],
        [1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1.]])


#### 3. Arithmetic Operations

In [16]:
x_tensor = torch.ones(4, 4)

# Transpose
x_T_tensor = x_tensor.T

# Matrix Multiplication
y1_tensor = x_tensor @ x_tensor.T
y2_tensor = x_tensor.matmul(x_tensor.T)

y3_tensor = torch.rand_like(y1_tensor)
torch.matmul(x_tensor, x_tensor.T, out=y3_tensor)


# Element-wise multiplication
z1_tensor = x_tensor * x_tensor
z2_tensor = x_tensor.mul(x_tensor)

z3_tensor = torch.rand_like(x_tensor)
torch.mul(x_tensor, x_tensor, out=z3_tensor)

tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]])

##### 3. In-place Operations

In [17]:
x_tensor = torch.ones(4, 4)

# Transpose
x_tensor.t_()

# Copy
y_tensor = torch.rand_like(x_tensor)
x_tensor.copy_(y_tensor)

tensor([[0.7934, 0.6413, 0.9885, 0.0340],
        [0.2890, 0.5230, 0.1286, 0.1077],
        [0.3840, 0.2738, 0.6143, 0.5598],
        [0.5460, 0.9349, 0.0589, 0.5723]])

### NumPy and Tensor
Tensors on the **CPU** and NumPy arrays can share memory locations, so modifying one will also affect 
the other.

In [18]:
x_tensor = torch.ones(5) 
x_np = x_tensor.numpy() # tensor to numpy
print(f"t: {x_tensor}")
print(f"n: {x_np}")

x_tensor.add_(1)

print(f"t: {x_tensor}")
print(f"n: {x_np}")

y_np = np.ones(5)
z_np = np.zeros(5)
y_tensor = torch.from_numpy(y_np) # numpy to tensor

np.add(y_np, 1, out=z_np)

print(f"t: {x_tensor}")
print(f"n: {x_np}")

t: tensor([1., 1., 1., 1., 1.])
n: [1. 1. 1. 1. 1.]
t: tensor([2., 2., 2., 2., 2.])
n: [2. 2. 2. 2. 2.]
t: tensor([2., 2., 2., 2., 2.])
n: [2. 2. 2. 2. 2.]


### Moving Tensor to GPU
It's always wise to check for GPU availability before performing any GPU operations. If a GPU is available, we can move our tensor to it.

In [19]:
x_tensor_gpu = x_tensor.to("cuda")

RuntimeError: No CUDA GPUs are available

A better approach is to set the default device before starting any computations.

In [21]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
y_tensor_gpu = y_tensor.to(device)

### Tensor attributes

In [22]:
print(f"Shape of tensor: {y_tensor.shape}")
print(f"Datatype of tensor: {y_tensor.dtype}")
print(f"Device tensor is stored on: {y_tensor.device}")

Shape of tensor: torch.Size([5])
Datatype of tensor: torch.float64
Device tensor is stored on: cpu


*Automatic differentiation* is a key feature that distinguishes tensors from NumPy arrays. This capability
is particularly useful in neural networks, where model weights are adjusted during backpropagation based 
on the gradient of the loss function with respect to each parameter. Tensors support automatic gradient 
computation for any computational graph. For example, consider the computational graph of a one-layer 
neural network:

In [23]:
x_tensor = torch.ones(5)  # input tensor
y_tensor = torch.zeros(3)  # expected output

w_tensor = torch.randn(5, 3, requires_grad=True)
b_tensor = torch.randn(3, requires_grad=True)

z_tensor = torch.matmul(x_tensor, w_tensor) + b_tensor

loss_tensor = torch.nn.functional.binary_cross_entropy_with_logits(z_tensor, y_tensor)
loss_tensor.backward()

print(w_tensor.grad)
print(b_tensor.grad)

tensor([[0.0752, 0.0121, 0.0481],
        [0.0752, 0.0121, 0.0481],
        [0.0752, 0.0121, 0.0481],
        [0.0752, 0.0121, 0.0481],
        [0.0752, 0.0121, 0.0481]])
tensor([0.0752, 0.0121, 0.0481])
